In [ ]:
from clickhouse_driver import Client
client = Client(host='localhost',password='bahar1375')
client.execute('SHOW DATABASES')

In [ ]:
client.execute(' DROP TABLE IF  EXISTS default.posts')
client.execute(' CREATE TABLE IF NOT EXISTS default.posts\
(\
    ID UUID,\
    sendTime DateTime,\
    senderName String,\
    senderUsername String,\
    keywords Array(String),\
    hashtags Array(String)\
) ENGINE = MergeTree()\
PARTITION BY toYYYYMMDD(sendTime)\
ORDER BY (sendTime);')

In [ ]:
client.execute(' DROP TABLE IF  EXISTS readings_queue ')

client.execute("CREATE TABLE readings_queue  (\
    message String\
  ) ENGINE = Kafka('185.236.37.254:9092', 'preProcessedData', 'group1', 'JSONAsString')");


In [ ]:
client.execute(' DROP TABLE IF  EXISTS posts_mv ')

client.execute("CREATE MATERIALIZED VIEW posts_mv TO posts AS SELECT \
    toUUID(JSONExtractString(message, 'id')) AS ID ,\
    JSONExtractString(message, 'sender_name') AS senderName ,\
    JSONExtractString(message, 'sender_username') AS senderUsername, \
    toDateTime(FROM_UNIXTIME(toUInt64(JSONExtractFloat(message, 'send_time')),'%Y-%m-%d %R:%S')) AS sendTime, \
    JSONExtractArrayRaw(message, 'hashtags') as hashtags, \
    JSONExtractArrayRaw(message, 'keywords') as keywords \
    FROM readings_queue")

In [ ]:
client.execute(' select  * from  default.posts limit 3')